#### 自然言語処理 入門
# word2vecによる自然言語処理

Bag of Wordsの問題点としてあげられるのは、__文の構造は全て無視して、どの単語が含まれているかだけを見ること__です。さらに、ベクトルの次元数は単語数と同じになるため、テキストによっては数百万次元などになってしまう可能性もあります。

例えば、以下のような単語があったとすると、Bag of Wordsではこのような表現の仕方になります。

![](https://s3.amazonaws.com/ai-standard/nlp-2-2.png)
（ちなみに、このようなベクトルのことをone-hotベクトルと呼びます。）
これだと、各ベクトル間の距離は一定になります。（距離＝$\sqrt 2$）

そこで先ほど挙げた問題点を解決するには、ベクトルの次元数を固定して、かつ__似ている単語は距離が近くなるようなベクトル表現__を実現するための手法が必要になります。それが今回扱う__word2vec__というものです。

__似ている単語は距離が近くなるようなベクトル表現__というもののイメージを持つために、図で解説します。
![](https://s3.amazonaws.com/ai-standard/nlp-2-1.png)
青文字で表されている物は、果物なので意味的には近いので、ベクトル空間上でも近い距離にいます。しかし、青文字で表されている物とオレンジ色で表されている物の間には意味的な近さは無いので、ベクトル空間上でも離れた位置にあります。


さらに、距離を近くすると同時に、単語の抽象度を上げていきます。そうすることによって、表現力豊かな分散表現を得ることができます。その結果、単語の加減算ができるようになります。

例えば、「King - Man + Woman」つまり「大様から男を引いて女を足した物は何か？」という計算が出来るようになります。結果は、queenです。
![](https://s3.amazonaws.com/ai-standard/nlp-2-3.png)

word2vecの目的は、以上のような__表現力の高い、単語の分散表現をつくること__です。

## word2vecの手法
今回は、word2vecで分散表現を求める、代表的な手法を２つ紹介します。

### Continuous Bag of Words (CBOW)
Continuous Bag of Words (CBOW)は、単語$w_t$の前後の単語（$w_{t-n}, ..., w_{t+n}$）を入力として、単語$w_t$を出力とするようなニューラルネットワークのことです。
![](https://s3.amazonaws.com/ai-standard/nlp-2-4.png)

単語の個数nはオプションで指定できます。デフォルトではn=5になっていることが多いです。

つまりCBOWの処理は、__文脈中の単語から対象単語が現れる条件付き確率を最大化すること__です。

### Skip-gram
Skip-gramは、CBOWと対照的に、単語$w_t$が与えられて、文脈中の1単語$w_{t+k}$を推定するアルゴリズムです。

学習過程では、入力層に単語$w_t$を入れ、正解データとして単語$w_{t+k}$を入れることを繰り返していきます。
![](https://s3.amazonaws.com/ai-standard/nlp-2-5.png)

つまり、Skip-gramは__出力層における周辺単語予測のエラー率の合計を最小化すること__です。


### CBOWとSkip-gram、どちらを使えばいいのか？
Skip-gramは学習データが少なくてもある程度の精度がでるとされています。 そして、CBoWよりもSkip-gramの方が実験では意味的な精度が高くなり、構文の精度ではあまり大差はないようです。

以上より、総合的に__Skip-gram__の方を使うのがよいでしょう。


## Gensimのインストール
word2vecをpythonで扱える__gensim__というライブラリをインストールします。

In [3]:
!pip install -U gensim

Requirement already up-to-date: gensim in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages/gensim-2.0.0-py3.5-macosx-10.7-x86_64.egg
Requirement already up-to-date: numpy>=1.3 in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages (from gensim)
    100% |████████████████████████████████| 16.1MB 41kB/s  eta 0:00:01
Requirement already up-to-date: six>=1.5.0 in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages (from gensim)
Requirement already up-to-date: smart_open>=1.2.1 in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages/smart_open-1.5.2-py3.5.egg (from gensim)
Requirement already up-to-date: boto>=2.32 in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages/boto-2.46.1-py3.5.egg (from smart_open>=1.2.1->gensim)
Requirement already up-to-date: bz2file in /Users/tsuruoka/.pyenv/versions/anaconda3-

### word2vecの学習済みモデルのダウンロード
今回は、googleが公開している学習済みモデルをつかって、word2vecを使っていきます。

[DLリンク](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)

「GoogleNews-vectors-negative300.bin.gz」というファイルをダウンロードして、解凍します。その後、そのファイルをこのnotebookと同じディレクトリに配置します。

※ このファイルは解凍前で1.65GB、解凍後は3.64GBになります。PCの残り容量にご注意ください。

In [7]:
from gensim.models import KeyedVectors

In [8]:
# googleのword2vecモデルを読み込む
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

__most_similarメソッド__を使うと、意味が近い単語を表示してくれます。

In [12]:
model.most_similar(['life'])

[('lives', 0.6027060747146606),
 ('chiseled_burnished_refined', 0.5310098528862),
 ('Interaction_enriches_your', 0.4961995482444763),
 ('Shanda_Interaction_enriches', 0.49457842111587524),
 ('Life', 0.4814741611480713),
 ('Fayaz_Wani_reports', 0.4636700749397278),
 ('lifestyle', 0.44911327958106995),
 ('joys_sorrows', 0.44626227021217346),
 ('humdrum_existence', 0.44414764642715454),
 ('earthly_existence', 0.44378799200057983)]

さらに、positive引数に単語をリストで入れると、単語どうしの足し算をします。

In [13]:
model.most_similar(positive=['bread', 'meat'])

[('meats', 0.7095532417297363),
 ('chicken', 0.6442906260490417),
 ('cheese', 0.64256751537323),
 ('stewing_steak', 0.6253078579902649),
 ('potatoes', 0.6224780678749084),
 ('sausages', 0.6213147044181824),
 ('presliced', 0.6200767755508423),
 ('breads', 0.6197592616081238),
 ('pasta', 0.6131477355957031),
 ('pork_sausages', 0.6109585762023926)]

先ほどの、「King - Man + Woman」つまり「王様から男を引いて女を足した物は何か？」という計算が出来るようになったかどうか、試してみましょう。
![](https://s3.amazonaws.com/ai-standard/nlp-2-3.png)

In [10]:
model.most_similar(positive=['king', 'women'], negative=['man'])

[('queen', 0.4827325940132141),
 ('queens', 0.4667814075946808),
 ('kumaris', 0.4653734564781189),
 ('kings', 0.45586392283439636),
 ('womens', 0.422832190990448),
 ('princes', 0.4176960587501526),
 ('Al_Anqari', 0.41725507378578186),
 ('concubines', 0.4011078178882599),
 ('monarch', 0.39624831080436707),
 ('monarchy', 0.39430150389671326)]

> ('queen', 0.4827325940132141)

見ての通り、意味を捉えた分散表現が学習できていることがわかりました。

## 単語の分散表現を取得
word2vecにより、学習した単語の分散表現を取得できるようになりました。表示の仕方は、pyhtonのディクショナリ形式です。

In [11]:
model['king']

array([  1.25976562e-01,   2.97851562e-02,   8.60595703e-03,
         1.39648438e-01,  -2.56347656e-02,  -3.61328125e-02,
         1.11816406e-01,  -1.98242188e-01,   5.12695312e-02,
         3.63281250e-01,  -2.42187500e-01,  -3.02734375e-01,
        -1.77734375e-01,  -2.49023438e-02,  -1.67968750e-01,
        -1.69921875e-01,   3.46679688e-02,   5.21850586e-03,
         4.63867188e-02,   1.28906250e-01,   1.36718750e-01,
         1.12792969e-01,   5.95703125e-02,   1.36718750e-01,
         1.01074219e-01,  -1.76757812e-01,  -2.51953125e-01,
         5.98144531e-02,   3.41796875e-01,  -3.11279297e-02,
         1.04492188e-01,   6.17675781e-02,   1.24511719e-01,
         4.00390625e-01,  -3.22265625e-01,   8.39843750e-02,
         3.90625000e-02,   5.85937500e-03,   7.03125000e-02,
         1.72851562e-01,   1.38671875e-01,  -2.31445312e-01,
         2.83203125e-01,   1.42578125e-01,   3.41796875e-01,
        -2.39257812e-02,  -1.09863281e-01,   3.32031250e-02,
        -5.46875000e-02,

単語の分散表現を得られることによって、これを特徴量としたデータセットをつくることも可能です。

<br>

### word2vecを使用した特徴量について
実際にテキストを特徴量にしようとしたときに躓く問題があります。それは、特徴量にしたい物は単語ではなく、一文の文章であったり、長い記事のテキストを特徴量に変換したい場面が多いでしょう。

word2vecは、単語をベクトルにすることは出来ますが、文章をベクトルにすることはできません。

仮に、文章をわかちがきして、その単語をベクトル化したものを連結させても、テキストの長さや単語数がバラバラなので、特徴量が一定の形にならないでしょう。


では、どうすればいいか。

まず一つ目には、特徴量の形を、単語数が最大のテキストに合わせ、それに満たない場合の値はすべて０で埋めるという方式です。CNNの時に勉強した、0-paddingのようなイメージです。

参考になる記事を載せておきます。

[畳み込みニューラルネットワークによる文書分類](http://qiita.com/ichiroex/items/f225f6d8eceb6796cc7e)

<br>

２つ目は、Doc2vecと呼ばれるアルゴリズムを使用する方法です。
Doc2vecはword2vecの派生形で、Doc＝文章をベクトル化してくれるアルゴリズムです。

こちらも、参考になる記事を載せておきます。

[Doc2Vecの仕組みとgensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html)


その他にも、次回やるRNNの特徴量として単語の分散表現を使う方法などがあるでしょう。